In [3]:
import numpy as np
import pickle
import json
import os
import sys
import copy
import sklearn.preprocessing
import models
from sklearn.neighbors import NearestNeighbors
import utils
from collections import Counter
import matplotlib.pyplot as plt

In [4]:
DATA_DIR = "../../data/fb15k/"
DUMP_FILE = "../dumps/fb15k_distmult_dump_norm.pkl"
MODEL_TYPE = "distmult"
mining_dir="mined"

In [3]:
train_data = utils.read_data(os.path.join(DATA_DIR,"train.txt"))
# dev_data = read_data(os.path.join(DATA_DIR,"valid.txt"))
# test_data = read_data(os.path.join(DATA_DIR,"test.txt"))

In [4]:
dump=utils.load_pickle(DUMP_FILE)
dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [5]:
model=models.TypedDM(DUMP_FILE)

In [6]:
mapped_train_data = utils.map_data(train_data,dump)
# mapped_dev_data = map_data(dev_data)
# mapped_test_data = map_data(test_data)

In [7]:
entity_to_rel=utils.get_ent_to_rel(mapped_train_data)

# Length 1 Rules

In [8]:
relation_count=len(dump['relation_to_id'])
nbrs = NearestNeighbors(n_neighbors=relation_count,metric=model.similarity_relembedding).fit(model.relation_matrix)
print(relation_count)
count_r,set_r=utils.get_relation_dict(mapped_train_data)
print(len(count_r),len(set_r))
support=5

1345
1345 1345


In [ ]:
rules_dict_1={}
count=0
count2=0
for r1 in range(relation_count):
    if(r1%100==0):
        print(r1)
    if(count_r[r1]<support):
        continue
    combined_rel=model.relation_matrix[r1].reshape((1,-1))
    distances, indices = nbrs.kneighbors(combined_rel)
    indices=indices[0]
    bool_arr=[False for i in range(relation_count)]
    for ind in indices:
        if ind!=r1:
            bool_arr[ind]=True
    cur_dict={}
    for pair_e1e2 in set_r[r1]:
        if pair_e1e2 not in entity_to_rel:
            print("Strange")
            continue
        for r2 in entity_to_rel[pair_e1e2]:
            count+=1
            if bool_arr[r2]==False:
                continue
            if r2 not in cur_dict:
                count2+=1
                cur_dict[r2]=0
            cur_dict[r2]+=1
    rules_dict_1[r1]=cur_dict            

In [ ]:
print(len(rules_dict_1))
print(count,count2)

# Length 2 Rules

In [9]:
index_head=utils.get_head_index(mapped_train_data)
count_r1_r2,set_r1_r2=utils.get_r1r2_count(mapped_train_data,index_head,get_set=True)

In [10]:
nbrs = NearestNeighbors(n_neighbors=100,metric=model.similarity_relembedding).fit(model.relation_matrix)
support=5

In [ ]:
count=0
rules_dict={}
for r1 in range(relation_count):
    if(r1%100==0):
        print(r1)
    for r2 in range(relation_count):
        if(r1==r2):
            continue
        pair=(r1,r2)
        if(pair not in count_r1_r2 or count_r1_r2[pair]<support):
            continue
        
        combined_rel=model.dot_relation(r1,r2).reshape((1,-1))
        distances, indices = nbrs.kneighbors(combined_rel)
        
        bool_arr=[False for i in range(relation_count)]
        for ind in indices[0]:
            if ind!=r1 and ind!=r2:
                bool_arr[ind]=True
        
        if bool_arr[r1]==True or bool_arr[r2]==True:
            print("Strange")
            print(indices[0],r1,r2)
        
        cur_dict={}
        entpair_lis=list(set_r1_r2[pair])
        for pair_e1e2 in entpair_lis:
            if pair_e1e2 not in entity_to_rel:
                continue
            for r in entity_to_rel[pair_e1e2]:
                if(bool_arr[r]==False):
                    continue
                if r not in cur_dict:
                    count+=1
                    cur_dict[r]=0
                cur_dict[r]+=1
        rules_dict[pair]=cur_dict
        

In [11]:
print(len(rules_dict))
print(relation_count)
print(count)

NameError: name 'rules_dict' is not defined

# Saving rules

In [ ]:
path1='rules_1_5.pkl'
path1=os.path.join(mining_dir,path1)
utils.dump_pickle(rules_dict_1,path1)

In [ ]:
path2='rules_2_5.pkl'
path2=os.path.join(mining_dir,path2)
utils.dump_pickle(rules_dict,path2)

## Saving auxilary data for rule2

In [ ]:
path='set_r1_r2.pkl'
path=os.path.join(mining_dir,path)
set_len_r1_r2={}
for r1r2 in set_r1_r2:
    set_len_r1_r2[r1r2]=len(set_r1_r2[r1r2])
utils.dump_pickle(set_len_r1_r2,path)

In [10]:
print(len(count_r1_r2))

56863


## Analyse 3 Rules

In [ ]:
min_sup=5
confidence=0

In [ ]:
rules_3=[]
count=0
count_dict={}

for r1r2 in set_r1_r2:
#     prit(r1r2)
    count+=1
    if(count%100==0):
        print(count)
    for e1e2 in set_r1_r2[r1r2]:
        e1=e1e2[0]
        e2=e1e2[1]
        # e1-r1-e-r2-e2
        count_conf_dict={}
#         count_dict={}
        
        # e1-r1-e-r2-e2-r3-e3
        if e2 not in index_head:
            continue
        for r3e3 in index_head[e2]:
            e3=r3e3[1]
            triplet=(r1r2[0],r1r2[1],r3e3[0])
            if triplet not in count_dict:
                count_dict[triplet]=0
                count_conf_dict[triplet]={}
            count_dict[triplet]+=1
            
#             pair=(e1,e3)
#             if pair not in entity_to_rel:
#                 continue
#             for rel in entity_to_rel[pair]:
#                 if rel not in count_conf_dict[triplet]:
#                     count_conf_dict[triplet][rel]=0
#                 count_conf_dict[triplet][rel]+=1
        
#         for r1r2r3,supp in count_dict.items():
#             if supp < min_sup:
#                 continue
#             for r4,conf in count_conf_dict[r1r2r3].items():
#                 rules_3.append((r1r2r3,r4,supp,conf))           
        

In [ ]:
print(len(count_dict))

In [ ]:
value_list=list(count_dict.values())
cnt=Counter(value_list)
cnt=sorted(cnt.items())
array=np.array(cnt)
print(array.shape)

In [ ]:
plt.plot(array[:,1])
plt.show()

In [ ]:
temp=np.cumsum(array[:,1])/np.sum(array[:,1])
print(array[temp[:]<0.5],temp[temp[:]<0.5])

## Rules Length 3

In [11]:
min_sup=10
confidence=0
r1r2_ent=utils.get_r1r2_e1e2_dict(mapped_train_data,index_head)

In [12]:
rules_3=[]
count=0

for r1r2 in set_r1_r2:
#     prit(r1r2)
    count+=1
    if(count%100==0):
        print(count)
        print(len(rules_3))
    for e1e2 in set_r1_r2[r1r2]:
        e1=e1e2[0]
        e2=e1e2[1]
        # e1-r1-e-r2-e2
        e=r1r2_ent[r1r2][e1e2]
        count_conf_dict={}
        count_dict={}
        
        # e1-r1-e-r2-e2-r3-e3
        if e2 not in index_head:
            continue
        for r3e3 in index_head[e2]:
            e3=r3e3[1]
            triplet=(r1r2[0],r1r2[1],r3e3[0])
            if triplet not in count_dict:
                count_dict[triplet]=0
                count_conf_dict[triplet]={}
            count_dict[triplet]+=1
            
            pair=(e1,e3)
            if pair not in entity_to_rel:
                continue
            for rel in entity_to_rel[pair]:
                if rel not in count_conf_dict[triplet]:
                    count_conf_dict[triplet][rel]=0
                count_conf_dict[triplet][rel]+=1
        
        for r1r2r3,supp in count_dict.items():
            if supp < min_sup:
                continue
            for r4,conf in count_conf_dict[r1r2r3].items():
                rules_3.append((r1r2r3,r4,supp,conf,e,e2))           
        

100
935260
200
3023981
300
4401260
400
4960717
500
6439043
600
6731787
700
9434380
800
11211250
900
11717621
1000
13017856
1100
14375488
1200
14696661
1300
15053997
1400
16122002
1500
16205939
1600
16404744
1700
16700553
1800
16899201
1900
17073703
2000
17209232
2100
17477970
2200
17988361
2300
18332280
2400
18572303
2500
19276606
2600
19573679
2700
19776506
2800
20400617
2900
21357261
3000
21531744
3100
21589905
3200
21906408
3300
21969091
3400
22199657
3500
22320602
3600
22474787
3700
22984864
3800
23089620
3900
23204611
4000
23460356
4100
23555731
4200
23831097
4300
24170202
4400
24294043
4500
24364817
4600
24418664
4700
24516127
4800
24565762
4900
24647439
5000
24690850
5100
24964904
5200
25411406
5300
25823408
5400
26029417
5500
26194820
5600
26251216
5700
26317584
5800
26360560
5900
26436169
6000
26482440
6100
26516650
6200
26622137
6300
26667245
6400
26692065
6500
26708873
6600
26740300
6700
26771025
6800
26795287
6900
26844315
7000
26908736
7100
26917213
7200
26987286
7300
2701

56100
31391770
56200
31391878
56300
31392406
56400
31392696
56500
31392890
56600
31393031
56700
31394258
56800
31394517


In [6]:
path_rule_3=os.path.join(mining_dir,"mined_3_ent.pkl")

In [13]:
utils.dump_pickle(rules_3,path_rule_3)

## Load Len 3 rules

In [7]:
rules_3=utils.load_pickle(path_rule_3)

In [11]:
rules_3.sort(key=lambda x:(x[3]*1.0)/x[2],reverse=True)

In [12]:
rules_3[0:10]

[((0, 108, 124), 124, 15, 15, 6321, 5324),
 ((0, 108, 97), 97, 15, 15, 6321, 5324),
 ((0, 108, 132), 132, 10, 10, 2795, 2146),
 ((0, 108, 323), 323, 19, 19, 2795, 2146),
 ((0, 108, 124), 124, 15, 15, 2795, 2146),
 ((0, 108, 97), 97, 21, 21, 2795, 2146),
 ((0, 108, 124), 124, 10, 10, 6321, 3041),
 ((0, 108, 97), 97, 12, 12, 6321, 3041),
 ((0, 108, 124), 124, 35, 35, 2795, 314),
 ((0, 108, 97), 97, 30, 30, 2795, 314)]